In [1]:
# register at https://nsgr.sdsc.edu:8444/restusers/register.action for username and password
CRA_USER = 'changeme'
PASSWORD = 'changeme'
# log in at https://nsgr.sdsc.edu:8444/restuser/login.action
# Tool names can be found at Developer->Documentation (Tools: How to Configure Specific Tools)
# create a new application at Developer->Application Management (Create New Application)
# save the Application Key for use in REST requests
KEY = 'neurontest1-AFD77DA4C0524E0395581CCC37DFF80B'
# for development version:
URL = 'https://nsgr.sdsc.edu:8444/cipresrest/v1'
# for production version:
#URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1'
TOOL = 'NEURON73_TG'


In [2]:
%%capture --no-stderr submitoutput

import os
import requests
import xml.etree.ElementTree
import time
import sys

os.popen('pwd').read()
os.popen('ls -ltr').read()
headers = {'cipres-appkey' : KEY}
payload = {'tool' : TOOL, 'metadata.statusEmail' : 'true', 'vparam.number_cores_' : 24, 'vparam.number_nodes_' : 2, 'vparam.runtime_' : 0.5, 'vparam.filename_': 'Batch.hoc', 'vparam.cmdlineopts_': '-c TSTOP=1'}
files = {'input.infile_' : open('/home/john/info/nsg/rest/JonesEtAl2009_r31.zip')}
#files = {'input.infile_' : data}
r = requests.post("%s/job/%s" % (URL, CRA_USER), auth=(CRA_USER, PASSWORD), data=payload, headers=headers, files=files)
#print(r.text)

root = xml.etree.ElementTree.fromstring(r.text)

sys.stderr.write("%s\n" % r.text)
sys.stderr.write("%s\n" % r.url)

for child in root:
    if child.tag == 'resultsUri':
        for urlchild in child:
            if urlchild.tag == 'url':
                outputuri = urlchild.text
    if child.tag == 'selfUri':
        for urlchild in child:
            if urlchild.tag == 'url':
                selfuri = urlchild.text
#print(outputuri,file=sys.stderr)
sys.stderr.write("%s\n" % outputuri)
#print(selfuri,file=sys.stderr)
sys.stderr.write("%s\n" % selfuri)

headers = {'cipres-appkey' : KEY}

#print('Waiting for job to complete',file=sys.stderr)
sys.stderr.write('Waiting for job to complete\n')
jobdone = 0
while jobdone == 0:
    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers)
    #print(r.text)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag == 'terminalStage':
            jobstatus = child.text
            if jobstatus == 'false':
                time.sleep(5)
                #print('.',file=sys.stderr,end='')
                sys.stderr.write('.')
            else:
                jobdone = 1
                #print('',file=sys.stderr,end='\n')
                sys.stderr.write('\n')
                break

#print('Job completion detected, getting download URIs...',file=sys.stderr)
sys.stderr.write('Job completion detected, getting download URIs...')
r = requests.get(outputuri,
                 headers= headers, auth=(CRA_USER, PASSWORD))
#print(r.text)
globaldownloadurilist = []
root = xml.etree.ElementTree.fromstring(r.text)
for child in root:
    if child.tag == 'jobfiles':
        for jobchild in child:
            if jobchild.tag == 'jobfile':
                for downloadchild in jobchild:
                    if downloadchild.tag == 'downloadUri':
                        for attchild in downloadchild:
                            if attchild.tag == 'url':
                                #print(attchild.text)
                                sys.stdout.write(attchild.text)
                                globaldownloadurilist.append(attchild.text)
                
#print('Download complete.  Run the next cell.',file=sys.stderr)
sys.stderr.write('Download complete.  Run the next cell.\n')

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<jobstatus>
    <selfUri>
        <url>https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-69D3EB3CCA964EC0BFE901742B6F0C31</url>
        <rel>jobstatus</rel>
        <title>NGBW-JOB-NEURON73_TG-69D3EB3CCA964EC0BFE901742B6F0C31</title>
    </selfUri>
    <jobHandle>NGBW-JOB-NEURON73_TG-69D3EB3CCA964EC0BFE901742B6F0C31</jobHandle>
    <jobStage>QUEUE</jobStage>
    <terminalStage>false</terminalStage>
    <failed>false</failed>
    <metadata>
        <entry>
            <key>statusEmail</key>
            <value>true</value>
        </entry>
    </metadata>
    <dateSubmitted>2016-04-07T17:54:12-07:00</dateSubmitted>
    <resultsUri>
        <url>https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-69D3EB3CCA964EC0BFE901742B6F0C31/output</url>
        <rel>results</rel>
        <title>Job Results</title>
    </resultsUri>
    <workingDirUri>
        <url>https://nsgr.sdsc.edu:8444/c

In [20]:
#submitoutput.show()
#print(submitoutput.stdout)
#print(globaldownloadurilist)
globaloutputdict = {}
for downloaduri in globaldownloadurilist:
    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
    #print(r.text)
    globaloutputdict[downloaduri] = r.text
for name in globaloutputdict.keys():
    continue_var = raw_input("display %s [Y/y]? " % name)
    if continue_var in ['Y','y']:
         #print(globaloutputdict[name])
         sys.stdout.write("%s\n" % globaloutputdict[name])   

display https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output/284 [Y/y]? n
display https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output/285 [Y/y]? n
display https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output/281 [Y/y]? n
display https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output/282 [Y/y]? n
display https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output/283 [Y/y]? n


In [11]:
#http://stackoverflow.com/questions/31804799/how-to-get-pdf-filename-with-python-requests
downloaduri = 'https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output/281'
r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
sys.stderr.write("%s\n" % r.headers)
import re
d = r.headers['content-disposition']
fname_list = re.findall("filename=(.+)", d)
for fname in fname_list:
    sys.stderr.write("%s\n" % fname)

{'Content-Disposition': 'attachment; filename=STDOUT', 'Transfer-Encoding': 'chunked', 'Expires': 'Wed, 31 Dec 1969 16:00:00 PST', 'Server': 'Apache-Coyote/1.1', 'Cache-Control': 'private', 'Date': 'Thu, 07 Apr 2016 23:36:05 GMT', 'Content-Type': 'text/plain'}
STDOUT


In [14]:
# download all output files

import re
#downloaduri = 'https://cipresrest.sdsc.edu/cipresrest/v1/job/kenneth/NGBW-JOB-RAXMLHPC8_REST_XSEDE-EF0E26B61D2E4C0C8D02499D8068D873/output/11496'
for downloaduri in globaldownloadurilist:
    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
    #sys.stderr.write("%s\n" % r.json())
    #r.content
    d = r.headers['content-disposition']
    filename_list = re.findall('filename=(.+)', d)
    for filename in filename_list:
        #http://docs.python-requests.org/en/master/user/quickstart/#raw-response-content
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

In [23]:
# get a list of jobs for user and app key, and terminalStage status

r = requests.get("%s/job/%s" % (URL,CRA_USER), auth=(CRA_USER, PASSWORD), headers=headers)
#print(r.text)

root = xml.etree.ElementTree.fromstring(r.text)
for child in root:
    if child.tag == 'jobs':
        for jobchild in child:
            if jobchild.tag == 'jobstatus':
                for statuschild in jobchild:
                    if statuschild.tag == 'selfUri':
                        for selfchild in statuschild:
                            if selfchild.tag == 'url':
                                #print(child)
                                joburi = selfchild.text
                                jobr = requests.get(joburi, auth=(CRA_USER, PASSWORD), headers=headers)
                                jobroot = xml.etree.ElementTree.fromstring(jobr.text)
                                for jobrchild in jobroot:
                                    if jobrchild.tag == 'terminalStage':
                                        jobstatus = jobrchild.text
                                        sys.stdout.write("job url: %s status terminalStage: %s\n" % (joburi,jobstatus))

job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-A97FBC66F92447CB8A00921F1310C35D status terminalStage: true
job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-BDEC62CAC45D485ABFDAFEF3680D42C6 status terminalStage: true
job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-6ACCB330DD374B25980086CE9E50C65A status terminalStage: true
job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-0E237A9603E24E43AF982EBC537D8488 status terminalStage: true
job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-82B87020280346C2ACF61AFFDB5F2F5E status terminalStage: true
job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-FA69B4C34D4E4C27B8A167567FF0107B status terminalStage: true
job url: https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99155EEC39094DF5AAC442A2B83C49A

In [19]:
# get information for a single job, print out raw XML

r = requests.get('https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B',
                 headers= headers, auth=(CRA_USER, PASSWORD))
print(r.text)


<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<jobstatus>
    <selfUri>
        <url>https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B</url>
        <rel>jobstatus</rel>
        <title>NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B</title>
    </selfUri>
    <jobHandle>NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B</jobHandle>
    <jobStage>COMPLETED</jobStage>
    <terminalStage>true</terminalStage>
    <failed>false</failed>
    <metadata>
        <entry>
            <key>statusEmail</key>
            <value>true</value>
        </entry>
    </metadata>
    <dateSubmitted>2016-04-07T15:37:13-07:00</dateSubmitted>
    <resultsUri>
        <url>https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-99E78C2A95DD4F9BA6711CE220C57D9B/output</url>
        <rel>results</rel>
        <title>Job Results</title>
    </resultsUri>
    <workingDirUri>
        <url>https://nsgr.sdsc.edu:844

In [22]:
# delete an old job
r = requests.delete('https://nsgr.sdsc.edu:8444/cipresrest/v1/job/kennethrest/NGBW-JOB-NEURON73_TG-E3594B6A48BF414FB20EA5B022350F19', auth=(CRA_USER, PASSWORD), headers=headers)
sys.stderr.write("%s\n" % r.text)